<h1> <u> Dataset Sources </u> </h1>

<ul> 
  <li> <a href="https://catalog.data.gov/dataset/farmers-markets-geographic-data"> Farmer Market Data </a> (Last Updated: Feb 4, 2018) </li>
  <li> <a href="https://catalog.data.gov/dataset/zip-code-data"> Zip Code Data </a> (Last Updated: Nov 7, 2017) </li>
</ul>

In [2]:
# Computational and Visualisation Packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from ggplot import *

# Pyspark Packages
from pyspark.sql import functions as F
from pyspark.sql.functions import col, desc
from pyspark.sql.types import *
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression, GeneralizedLinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

In [3]:
us_tax = spark.read.csv('/databricks-datasets/data.gov/irs_zip_code_data/data-001/2013_soi_zipcode_agi.csv', header=True)
farmer_market = spark.read.csv('/databricks-datasets/data.gov/farmers_markets_geographic_data/data-001/market_data.csv', header=True)

In [4]:
display (us_tax.sample(False, 0.3), 100)

STATEFIPS,STATE,zipcode,agi_stub,N1,MARS1,MARS2,MARS4,PREP,N2,NUMDEP,A00100,N02650,A02650,N00200,A00200,N00300,A00300,N00600,A00600,N00650,A00650,N00700,A00700,N00900,A00900,N01000,A01000,N01400,A01400,N01700,A01700,SCHF,N02300,A02300,N02500,A02500,N26270,A26270,N02900,A02900,N03220,A03220,N03300,A03300,N03270,A03270,N03150,A03150,N03210,A03210,N03230,A03230,N03240,A03240,N04470,A04470,A00101,N18425,A18425,N18450,A18450,N18500,A18500,N18300,A18300,N19300,A19300,N19700,A19700,N04800,A04800,N05800,A05800,N09600,A09600,N07100,A07100,N07300,A07300,N07180,A07180,N07230,A07230,N07240,A07240,N07220,A07220,N07260,A07260,N09400,A09400,N10600,A10600,N59660,A59660,N59720,A59720,N11070,A11070,N10960,A10960,N06500,A06500,N10300,A10300,N85330,A85330,N85300,A85300,N11901,A11901,N11902,A11902
01,AL,00000,1,870380.0000,488030.0000,122290.0000,247000.0000,500770.0000,1452580.0000,571240.0000,11255896.000,870380.0000,11444868.000,700700.0000,8889326.0000,103290.0000,77952.0000,46870.0000,75071.0000,40890.0000,47416.0000,15650.0000,6538.0000,146240.0000,824487.0000,37970.0000,23583.0000,38400.0000,221790.0000,111060.0000,1066291.0000,8800.0000,49720.0000,187559.0000,35370.0000,62791.0000,8980.0000,10323.0000,155190.0000,186574.0000,3300.0000,747.0000,140.0000,487.0000,8950.0000,33584.0000,2540.0000,6212.0000,20760.0000,17533.0000,5900.0000,16956.0000,30.0000,2.0000,57090.0000,794815.0000,884758.0000,23440.0000,22991.0000,25940.0000,20686.0000,29140.0000,25144.0000,54970.0000,84317.0000,27140.0000,151005.0000,42500.0000,111909.0000,338560.0000,1874627.0000,333040.0000,196535.0000,0.0000,0.0000,108740.0000,37220.0000,2400.0000,56.0000,8850.0000,2899.0000,33010.0000,16969.0000,32080.0000,5457.0000,33530.0000,10160.0000,2650.0000,677.0000,117420.0000,152943.0000,807980.0000,2277816.0000,399710.0000,1174659.0000,371280.0000,1057938.0000,258200.0000,327115.0000,80350.0000,76412.0000,255750.0000,159189.0000,371450.0000,318777.0000,0.0000,0.0000,0.0000,0.0000,59580.0000,44367.0000,767170.0000,2005593.0000
01,AL,35004,3,910.0000,290.0000,490.0000,110.0000,450.0000,2020.0000,620.0000,55761.0000,910.0000,56170.0000,840.0000,47285.0000,250.0000,185.0000,120.0000,380.0000,100.0000,283.0000,310.0000,178.0000,120.0000,584.0000,80.0000,139.0000,70.0000,820.0000,190.0000,3420.0000,0.0000,40.0000,152.0000,150.0000,2058.0000,30.0000,492.0000,260.0000,409.0000,40.0000,10.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,140.0000,138.0000,0.0000,0.0000,0.0000,0.0000,430.0000,6464.0000,26501.0000,390.0000,1022.0000,40.0000,60.0000,370.0000,231.0000,430.0000,1396.0000,360.0000,2265.0000,350.0000,1137.0000,910.0000,36244.0000,900.0000,5051.0000,0.0000,0.0000,410.0000,629.0000,40.0000,2.0000,90.0000,47.0000,80.0000,98.0000,60.0000,8.0000,300.0000,448.0000,30.0000,9.0000,70.0000,124.0000,890.0000,6261.0000,0.0000,0.0000,0.0000,0.0000,40.0000,35.0000,60.0000,52.0000,870.0000,4422.0000,880.0000,4583.0000,0.0000,0.0000,0.0000,0.0000,180.0000,327.0000,720.0000,1974.0000
01,AL,35005,3,470.0000,140.0000,230.0000,100.0000,290.0000,1040.0000,340.0000,28531.0000,470.0000,28664.0000,440.0000,23743.0000,110.0000,95.0000,50.0000,118.0000,40.0000,106.0000,150.0000,92.0000,50.0000,233.0000,20.0000,22.0000,40.0000,543.0000,110.0000,2237.0000,0.0000,30.0000,87.0000,100.0000,1267.0000,50.0000,918.0000,100.0000,132.0000,30.0000,7.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,50.0000,39.0000,0.0000,0.0000,0.0000,0.0000,250.0000,4129.0000,14931.0000,210.0000,542.0000,40.0000,76.0000,210.0000,151.0000,250.0000,848.0000,200.0000,1121.0000,220.0000,1073.0000,470.0000,17954.0000,470.0000,2472.0000,0.0000,0.0000,190.0000,286.0000,0.0000,0.0000,30.0000,19.0000,40.0000,60.0000,30.0000,3.0000,130.0000,185.0000,30.0000,19.0000,40.0000,71.0000,470.0000,3081.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,40.0000,35.0000,450.0000,2186.0000,460.0000,2283.0000,0.0000,0.0000,0.0000,0.0000,110.0000,210.0000,360.0000,1002.0000
01,AL,35005,6,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,

In [5]:
display (farmer_market.sample(False, 0.3), 100)

FMID,MarketName,Website,Facebook,Twitter,Youtube,OtherMedia,street,city,County,State,zip,Season1Date,Season1Time,Season2Date,Season2Time,Season3Date,Season3Time,Season4Date,Season4Time,x,y,Location,Credit,WIC,WICcash,SFMNP,SNAP,Organic,Bakedgoods,Cheese,Crafts,Flowers,Eggs,Seafood,Herbs,Vegetables,Honey,Jams,Maple,Meat,Nursery,Nuts,Plants,Poultry,Prepared,Soap,Trees,Wine,Coffee,Beans,Fruits,Grains,Juices,Mushrooms,PetFood,Tofu,WildHarvested,updateTime
1010376,18th Street Farmers Market,null,https://www.facebook.com/groups/631614906915563/,null,null,null,825 18th Street,Charleston,Coles,Illinois,61920,06/07/2014 to 10/04/2014,Sat: 8:00 AM-12:00 PM;,null,null,null,null,null,null,-88.163254,39.490737,Local government building grounds,N,Y,N,Y,Y,Y,Y,N,Y,Y,N,N,Y,Y,Y,Y,N,N,N,N,Y,N,Y,Y,Y,N,N,N,Y,N,N,N,N,N,N,8/3/2014 12:22:27 PM
1000709,20th Annual Highlands Business Partnership Farmers Market,http://www.highlandsnj.com,https://www.facebook.com/pages/Highlands-Business-Partnership-Highlands-NJ/165486543524770,null,null,null,71 Waterwitch Avenue,Highlands,Monmouth,New Jersey,07732,06/28/2014 to 11/01/2014,Sat: 8:30 AM-2:00 PM;,null,null,null,null,null,null,-73.994358,40.404837,Local government building grounds,Y,Y,Y,Y,Y,-,Y,Y,Y,Y,Y,Y,Y,Y,Y,N,N,N,N,Y,Y,N,N,Y,N,N,N,N,Y,N,N,N,N,N,N,4/9/2014 10:45:36 AM
1010873,21st Street Farmers Market,null,null,null,null,null,SW 21st and Oakley,Topeka,Shawnee,Kansas,66604,06/05/2015 to 10/30/2015,Fri: 7:30 AM-12:00 PM;,null,null,null,null,null,null,-95.714514,39.028184,null,Y,N,N,Y,N,Y,Y,N,Y,Y,Y,N,Y,Y,Y,Y,N,N,N,Y,Y,N,Y,Y,N,N,N,N,Y,Y,N,N,N,N,N,3/2/2015 10:46:33 AM
1010966,26th and Allegheny,http://thefoodtrust.org/farmers-markets/market/26th-Allegheny,null,null,null,null,26th Street and W Allegheny Avenue,Philadelphia,Philadelphia,Pennsylvania,19129,06/01/2015 to 11/15/2015,Wed: 1:00 PM-5:00 PM;,null,null,null,null,null,null,-75.1722883,40.0040946,null,Y,Y,N,Y,Y,N,N,N,N,N,N,N,N,Y,N,N,N,N,N,N,N,N,N,N,N,N,N,N,Y,N,N,N,N,N,N,3/24/2015 3:17:20 PM
1010965,4th and Lehigh Farmers' Market,http://thefoodtrust.org/farmers-markets/market/fairhill-square,null,null,null,null,N 4th Street and W. Lehigh Avenue,Philadelphia,Philadelphia,Pennsylvania,19133,06/01/2015 to 11/22/2015,Tue: 1:00 PM-5:00 PM;,null,null,null,null,null,null,-75.138654,39.991748,null,Y,Y,N,Y,Y,N,N,N,N,N,N,N,N,Y,N,N,N,N,N,N,N,N,N,N,N,N,N,N,Y,N,N,N,N,N,N,3/24/2015 3:11:40 PM
1000060,57th Street Greenmarket,http://www.grownyc.org,https://www.facebook.com/ManhattanGreenmarkets,null,null,null,W 57 St. & 9 Ave,New York,New York,New York,10019,04/20/2013 to 12/21/2013,Wed: 8:00 AM-5:00 PM;Sat: 8:00 AM-5:00 PM;,null,null,null,null,null,null,-73.985839,40.768159,Other,Y,Y,Y,Y,Y,N,Y,Y,N,N,Y,N,N,Y,N,N,N,Y,N,N,N,Y,N,N,N,N,N,N,Y,N,N,N,N,N,N,7/24/2013 11:06:00 AM
1005306,58th and Chester Farmers' Market,http://thefoodtrust.org/farmers-markets/market/58th-chester,null,null,null,null,58th Street and Chester Avenue,Philadelphia,Philadelphia,Pennsylvania,19143,06/01/2015 to 11/01/2015,Wed: 2:00 PM-6:00 PM;,null,null,null,null,null,null,-75.228303,39.935721,null,Y,Y,N,Y,Y,N,N,N,N,N,N,N,N,Y,N,N,N,N,N,N,N,N,N,N,N,N,N,N,Y,N,N,N,N,N,N,3/24/2015 2:35:04 PM
1002678,61st Street Farmers Market,http://www.experimentalstation.org/farmers-market,https://www.facebook.com/61stStreetFarmersMarket,null,null,null,6100 S Blackstone,Chicago,Cook,Illinois,60637,01/25/2014 to 04/26/2014,Sat: 9:00 AM-2:00 PM;,05/17/2014 to 10/25/2014,Sat: 9:00 AM-2:00 PM;,11/01/2014 to 12/13/2014,Sat: 9:00 AM-2:00 PM;,null,null,-87.590793,41.784342,Closed-off public street,Y,Y,Y,Y,Y,Y,Y,Y,N,Y,Y,N,Y,Y,Y,Y,Y,Y,N,Y,Y,Y,Y,N,N,N,N,N,Y,N,N,N,N,N,N,4/23/2014 2:04:20 PM
1000061,79th Street Greenmarket,http://www.grownyc.org,https://www.facebook.com/ManhattanGreenmarkets,null,null,null,Columbus - W 78 & 81 Sts.,New York,New York,New York,10024,01/01/2013 to 12/31/2013,Sun: 8:00 AM-5:00 PM;,null,null,null,null,null,null,-73.9757,40.7818,Other,Y,Y,Y,Y,Y,N,Y,Y,N,Y,Y,Y,Y,Y,Y,Y,Y,Y,N,N,Y,Y,Y,N,N,N,N,N,Y,N,N,N,N,N,N,7/2

In [6]:
display (us_tax.describe())

summary,STATEFIPS,STATE,zipcode,agi_stub,N1,MARS1,MARS2,MARS4,PREP,N2,NUMDEP,A00100,N02650,A02650,N00200,A00200,N00300,A00300,N00600,A00600,N00650,A00650,N00700,A00700,N00900,A00900,N01000,A01000,N01400,A01400,N01700,A01700,SCHF,N02300,A02300,N02500,A02500,N26270,A26270,N02900,A02900,N03220,A03220,N03300,A03300,N03270,A03270,N03150,A03150,N03210,A03210,N03230,A03230,N03240,A03240,N04470,A04470,A00101,N18425,A18425,N18450,A18450,N18500,A18500,N18300,A18300,N19300,A19300,N19700,A19700,N04800,A04800,N05800,A05800,N09600,A09600,N07100,A07100,N07300,A07300,N07180,A07180,N07230,A07230,N07240,A07240,N07220,A07220,N07260,A07260,N09400,A09400,N10600,A10600,N59660,A59660,N59720,A59720,N11070,A11070,N10960,A10960,N06500,A06500,N10300,A10300,N85330,A85330,N85300,A85300,N11901,A11901,N11902,A11902
count,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740,166740
mean,29.673299748110832,null,48853.603274559195,3.5,1724.3668585822238,797.117608252369,634.7534484826676,260.0802446923354,948.5900803646396,3409.575806645076,1157.9081204270121,109623.8864099796,1724.365299268322,111240.51847187238,1444.2636439966416,76348.68934868657,522.4330094758307,992.0911598896486,319.82745591939545,2252.5905961376993,293.9652752788773,1625.1595178121627,253.96671464555595,317.91415377234017,273.9436847786974,3718.659985606333,275.14891447762983,4792.704677941706,156.1160489384671,2487.5708408300347,323.53118627803764,7410.385498380712,20.062852344968213,112.90602135060573,635.469617368358,217.56561113110232,2874.3542281396185,94.58474271320618,6376.52308984047,438.60207508696175,1584.5677941705649,43.44488425092959,10.974319299508217,9.727779776898165,231.83918675782655,44.427491903562434,301.3600815641118,28.95034184958618,136.19686937747392,133.29980808444284,134.90089960417416,19.669425452800766,43.69257526688257,5.721122706009356,78.61055535564351,532.6807004917836,14033.02644836272,70168.89840470193,388.3840110351445,3553.5931510135542,117.56729039222742,189.4551517332374,453.23755547559074,2078.732757586662,528.713626004558,6000.9269281516135,394.7554276118508,3491.1308504258127,437.86038143216985,2149.8618567830154,1313.7518291951542,75191.81502938707,1302.9902243013075,15124.835552356963,45.67434328895286,310.10578145615926,539.978109631762,703.7870337051697,84.255547559074,125.17783375314862,73.48482667626244,39.7027587861341,119.11137099676142,123.94679141177882,84.69773299748111,14.667002518891687,268.0231498140818,323.36886170085165,35.38850905601535,15.941465755067771,219.88473071848387,643.5925872616049,1637.714705529567,18176.12246611491,338.19527407940507,802.207820558954,294.73647595058173,695.8103394506417,241.82409739714527,323.410759265923,119.35228499460237,105.71308624205349,1164.1902962696413,14457.736631881971,1270.9345687897326,15389.845903802327,32.74637159649754,69.89030826436368,35.88557034904642,170.04715125344848,313.5573947463116,1583.3137579465035,1327.395825836632,3881.419395465995
stddev,15.109923412643349,null,27140.030935908577,1.707830248905018,36918.28626998589,21282.966256269487,11364.895814177335,7082.007826205272,20656.559708495715,67536.41774568192,24217.127341786672,2328809.8855352397,36918.24240918538,2363637.2436273536,29987.272095635224,1518760.3201167285,9559.30034056247,33845.875050314

In [7]:
display (farmer_market.describe())

summary,FMID,MarketName,Website,Facebook,Twitter,Youtube,OtherMedia,street,city,County,State,zip,Season1Date,Season1Time,Season2Date,Season2Time,Season3Date,Season3Time,Season4Date,Season4Time,x,y,Location,Credit,WIC,WICcash,SFMNP,SNAP,Organic,Bakedgoods,Cheese,Crafts,Flowers,Eggs,Seafood,Herbs,Vegetables,Honey,Jams,Maple,Meat,Nursery,Nuts,Plants,Poultry,Prepared,Soap,Trees,Wine,Coffee,Beans,Fruits,Grains,Juices,Mushrooms,PetFood,Tofu,WildHarvested,updateTime
count,8518,8518,5003,3344,845,134,434,8228,8474,7930,8517,7528,5026,5130,384,375,67,65,7,7,8487,8487,3698,8518,8518,8518,8518,8518,8518,8518,8518,8518,8518,8518,8518,8518,8518,8518,8518,8518,8518,8518,8518,8518,8518,8518,8518,8518,8518,8518,8518,8518,8518,8518,8518,8518,8518,8518,8518
mean,1009019.2689598497,null,null,null,null,null,null,98383.0,null,null,null,47501.76667998402,null,null,null,null,null,null,null,null,-91.0307326892509,39.17351085931251,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2009.936057510405
stddev,57015.38311039544,null,null,null,null,null,null,NaN,null,null,null,30650.047052285274,null,null,null,null,null,null,null,null,17.52819710985125,5.284537517011239,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1.4734842504100931
min,1000001,"""Badseed """"Funky"""" Farmers' Market""",http:// jasper.agrilife.org/jasper-county-farmers-market/,www.facebook.com/pages/Haddon-Heights-Farmers-Market/219172298144851,no twitter,Buena Vista,http://instagram.com/forsythfarmersmarket,1102 McConnell Road,Dayton,ALAMEDA,Alabama,00000,01/01/2011 to 12/31/2011,Fri: 10:00 AM-1:00 PM;,01/01/2014 to 04/01/2014,Fri: 11:30 AM-1:00 PM;,01/01/2012 to 05/31/2012,Fri: 9:00 AM-1:00 PM;,01/03/2014 to 05/30/2014,Fri: 9:00 AM-1:00 PM;,-88.174004,17.7099,Closed-off public street,N,N,N,N,N,-,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,1/1/2015 10:12:55 PM
max,2000036,wilbraham winter farmers market,https://www.msuextension.org/Richland/,yes,yes,youtube.com/c/4mrmarketOrg,www.yelp.com/biz/battens-farmers-market-davie,Broadway & South Third Streets,woodstock,midland,Wyoming,n/a,Start Date 1/1/13 to End Date 12/31/13,sun:TBD - TBD;,October to March,sun:10:00 AM - 2:00 PM;,September to October,Wed: 8:00 AM-12:00 PM;Sat: 8:00 AM-12:00 PM;,11/07/2015 to 11/28/2015,Thu: 3:00 PM-7:00 PM;,-99.9987,64.86275,Private business parking lot,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,May 30 2010 7:17PM


In [8]:
us_tax_filtered = us_tax\
    .withColumn("zipcode", (col('zipcode')/10).cast('int'))\
    .withColumn("single_returns", col('mars1').cast('int'))\
    .withColumn("joint_returns", col('mars2').cast('int'))\
    .withColumn("numdep", col('numdep').cast('int'))\
    .withColumn("total_income_amount", col('A02650').cast('double'))\
    .withColumn("taxable_interest_amount", col('A00300').cast('double'))\
    .withColumn("net_capital_gains", col('a01000').cast('double'))\
    .withColumn("biz_net_income", col('a00900').cast('double'))

us_tax_filtered = us_tax_filtered [['state', 'zipcode', 'single_returns', 'joint_returns', 'numdep', 'total_income_amount', 'taxable_interest_amount', 'net_capital_gains', 'biz_net_income']]
us_tax_filtered.createOrReplaceTempView('us_tax_filtered')

# Average Taxable Interest across the US States
display (us_tax_filtered)

state,zipcode,single_returns,joint_returns,numdep,total_income_amount,taxable_interest_amount,net_capital_gains,biz_net_income
AL,0,488030,122290,571240,1.1444868E7,77952.0,23583.0,824487.0
AL,0,195840,155230,383240,1.7810952E7,81216.0,54639.0,252768.0
AL,0,72710,146880,189340,1.6070153E7,80627.0,84137.0,259836.0
AL,0,24860,126480,134370,1.4288572E7,71086.0,105947.0,214668.0
AL,0,16930,168170,177800,2.605392E7,149150.0,404166.0,567439.0
AL,0,3530,42190,48270,2.0752068E7,271416.0,1569967.0,822565.0
AL,3500,950,260,710,19851.0,183.0,4.0,1657.0
AL,3500,590,410,860,49338.0,172.0,54.0,788.0
AL,3500,290,490,620,56170.0,185.0,139.0,584.0
AL,3500,90,490,530,52977.0,89.0,173.0,339.0


In [9]:
display(us_tax_filtered.filter('zipcode NOT IN (0000, 9999)').groupby('zipcode').agg(F.sum('net_capital_gains').alias('capital_gains'))\
        .sort(desc('capital_gains')).limit(40))

zipcode,capital_gains
1002,1.2443093E7
1001,8606053.0
9430,5976580.0
9402,4667696.0
1000,4110129.0
683,3964177.0
9411,3929969.0
6061,3851723.0
1012,2426170.0
7702,2186449.0


In [10]:
display(us_tax_filtered.filter('zipcode NOT IN (0000, 9999)').groupby('zipcode').agg(F.sum('biz_net_income').alias('business_net_income'), \
     F.sum('net_capital_gains').alias('capital_gains'), (F.sum('net_capital_gains') + F.sum('biz_net_income')).alias('capital_and_business_income'))\
      .sort(desc('capital_and_business_income')).limit(40))

zipcode,business_net_income,capital_gains,capital_and_business_income
1002,1866926.0,1.2443093E7,1.4310019E7
1001,1491999.0,8606053.0,1.0098052E7
9430,349156.0,5976580.0,6325736.0
9402,681523.0,4667696.0,5349219.0
9411,1027768.0,3929969.0,4957737.0
1000,685593.0,4110129.0,4795722.0
6061,793338.0,3851723.0,4645061.0
683,277152.0,3964177.0,4241329.0
1012,331281.0,2426170.0,2757451.0
7702,541319.0,2186449.0,2727768.0


In [11]:
display(farmer_market.groupBy('state').agg(F.count(F.lit(1)).alias('Total Markets')))

state,Total Markets
Utah,41
Hawaii,97
Minnesota,189
Ohio,321
Oregon,171
Arkansas,100
Texas,205
North Dakota,65
null,1
Pennsylvania,303


In [12]:
us_tax_cumm = us_tax_filtered.groupBy('zipcode').sum()
farmer_market_cleaned = farmer_market.withColumn("zipcode", (col("zip")/10)).groupby('zipcode').count()\
                        .select(col('count').cast('double').alias('count'), col('zipcode').alias('zip'))
us_tax_expanded = farmer_market_cleaned.join(us_tax_cumm, farmer_market_cleaned.zip == us_tax_cumm.zipcode, how='outer').na.fill(0)
us_tax_expanded = us_tax_expanded.select(col('count'), col('zip'), col('zipcode'), col('sum(zipcode)').alias('sum_zipcode'), col('sum(single_returns)').alias('sum_single_returns'), col('sum(joint_returns)').alias('sum_joint_returns'), col('sum(numdep)').alias('sum_numdep'), col('sum(total_income_amount)').alias('sum_total_income_amount'), col('sum(taxable_interest_amount)').alias('sum_taxable_interest_amount'), col('sum(net_capital_gains)').alias('sum_net_capital_gains'), col('sum(biz_net_income)').alias('sum_biz_net_income'))

# Bringing featured columns as a single columns
feature_columns = ['sum_zipcode', 'sum_single_returns', 'sum_joint_returns', 'sum_numdep', 'sum_total_income_amount', 'sum_taxable_interest_amount', 'sum_net_capital_gains', 'sum_biz_net_income']
us_tax_assembler_model = VectorAssembler(inputCols=feature_columns, outputCol='features')
us_tax_expanded_prepared = us_tax_assembler_model.transform(us_tax_expanded)

# Splitting the data into training and testing set
train, test = us_tax_expanded_prepared.randomSplit([.65, .25])

In [13]:
lrReg = LinearRegression (maxIter=50, regParam=0.3, labelCol='count', elasticNetParam=0.4)
lrModel = lrReg.fit (train)
lrModelSummary = lrModel.summary
print ("Computed Coefficients = ", lrModel.coefficients)
print ("Computed Intercepts = ", lrModel.intercept)
print ("Objective History = ", lrModelSummary.objectiveHistory)
print ("Mean Absolute Error, MAE = ", lrModelSummary.meanAbsoluteError)
print ("RMSE = {0}".format(lrModelSummary.rootMeanSquaredError))
print ("R^2 = {0}".format(lrModelSummary.r2))

Computed Coefficients = [-1.5480759019584317e-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
Computed Intercepts = 0.7877943753218855
Objective History = [0.5000000000000001, 0.49343133316304605, 0.4704443671349117, 0.4664329193322373, 0.4664287728447907, 0.4664284212623797, 0.4664283659392503, 0.4664283572338983, 0.4664283558640706, 0.46642835564852186, 0.46642835561460416, 0.46642835560926715, 0.4664283556085139, 0.46642835560831264]
Mean Absolute Error, MAE = 0.5165007778384498
RMSE = 0.6819315027330547
R^2 = 0.1554351481826839

In [14]:
print (lrModel.explainParams())

aggregationDepth: suggested depth for treeAggregate (>= 2) (default: 2)
elasticNetParam: the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty (default: 0.0, current: 0.4)
epsilon: The shape parameter to control the amount of robustness. Must be > 1.0. (default: 1.35)
featuresCol: features column name (default: features)
fitIntercept: whether to fit an intercept term (default: True)
labelCol: label column name (default: label, current: count)
loss: The loss function to be optimized. Supported options: squaredError, huber. (Default squaredError) (default: squaredError)
maxIter: maximum number of iterations (>= 0) (default: 100, current: 50)
predictionCol: prediction column name (default: prediction)
regParam: regularization parameter (>= 0) (default: 0.0, current: 0.3)
solver: The solver algorithm for optimization. Supported options: auto, normal, l-bfgs. (Default auto) (default: auto)
standardization: whether to standardize the training features before fitting the model (default: True)
tol: the convergence tolerance for iterative algorithms (>= 0) (default: 1e-06)
weightCol: weight column name. If this is not set or empty, we treat all instance weights as 1.0 (undefined)

In [15]:
display (lrModel, test, "fittedVsResiduals")

fitted values,residuals
0.7533342057442908,-0.7533342057442908
0.7877943753218855,0.21220562467811455
0.7877943753218855,0.21220562467811455
0.5838570483014895,-0.5838570483014895
0.7877943753218855,0.21220562467811455
0.576463437793736,-0.576463437793736
0.595012483251002,-0.595012483251002
0.7877943753218855,0.21220562467811455
0.7877943753218855,0.21220562467811455
0.7877943753218855,0.21220562467811455


In [16]:
display (lrModelSummary.residuals)

residuals
-0.7601519320165157
-0.7292213754953862
-0.686215826938981
-0.5205097823933504
-0.7108488106909435
-0.5202868594634684
-0.41224354611398556
-0.6789708317178156
-0.23162953063249536
-0.4400810469830021


In [17]:
lrPredictions = lrModel.transform(test)
lrEvaluator = RegressionEvaluator(labelCol="count", predictionCol="prediction", metricName="rmse")
rmse = lrEvaluator.evaluate(lrPredictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

Root Mean Squared Error (RMSE) on test data = 18.1542

In [18]:
display(lrPredictions, 1000)

count,zip,zipcode,sum_zipcode,sum_single_returns,sum_joint_returns,sum_numdep,sum_total_income_amount,sum_taxable_interest_amount,sum_net_capital_gains,sum_biz_net_income,features,prediction
0.0,0.0,305,10980,25040,22560,29630,3711733.0,16160.0,68018.0,128687.0,"List(1, 8, List(), List(10980.0, 25040.0, 22560.0, 29630.0, 3711733.0, 16160.0, 68018.0, 128687.0))",0.7707965019183819
0.0,0.0,2862,120204,9230,10660,15950,1083118.0,6959.0,20019.0,35787.0,"List(1, 8, List(), List(120204.0, 9230.0, 10660.0, 15950.0, 1083118.0, 6959.0, 20019.0, 35787.0))",0.6017094596028741
0.0,0.0,5983,179490,1660,2280,2550,244294.0,2011.0,7647.0,6364.0,"List(1, 8, List(), List(179490.0, 1660.0, 2280.0, 2550.0, 244294.0, 2011.0, 7647.0, 6364.0))",0.5099302316793666
0.0,0.0,6765,202950,1160,1050,1280,123276.0,1366.0,4671.0,4683.0,"List(1, 8, List(), List(202950.0, 1160.0, 1050.0, 1280.0, 123276.0, 1366.0, 4671.0, 4683.0))",0.4736123710194217
0.0,0.0,7313,263268,19740,14960,28140,2222300.0,13873.0,33345.0,58180.0,"List(1, 8, List(), List(263268.0, 19740.0, 14960.0, 28140.0, 2222300.0, 13873.0, 33345.0, 58180.0))",0.38023552876509303
1.0,211.6,0,0,0,0,0,0.0,0.0,0.0,0.0,"List(0, 8, List(), List())",0.7877943753218855
1.0,875.2,0,0,0,0,0,0.0,0.0,0.0,0.0,"List(0, 8, List(), List())",0.7877943753218855
1.0,2350.4,0,0,0,0,0,0.0,0.0,0.0,0.0,"List(0, 8, List(), List())",0.7877943753218855
1.0,2815.0,2815,50670,8510,7970,15820,916019.0,5718.0,11020.0,22524.0,"List(1, 8, List(), List(50670.0, 8510.0, 7970.0, 15820.0, 916019.0, 5718.0, 11020.0, 22524.0))",0.7093533693696518
1.0,4968.6,0,0,0,0,0,0.0,0.0,0.0,0.0,"List(0, 8, List(), List())",0.7877943753218855


In [19]:
display(lrPredictions)

count,zip,zipcode,sum_zipcode,sum_single_returns,sum_joint_returns,sum_numdep,sum_total_income_amount,sum_taxable_interest_amount,sum_net_capital_gains,sum_biz_net_income,features,prediction
0.0,0.0,305,10980,25040,22560,29630,3711733.0,16160.0,68018.0,128687.0,"List(1, 8, List(), List(10980.0, 25040.0, 22560.0, 29630.0, 3711733.0, 16160.0, 68018.0, 128687.0))",0.7707965019183819
0.0,0.0,2862,120204,9230,10660,15950,1083118.0,6959.0,20019.0,35787.0,"List(1, 8, List(), List(120204.0, 9230.0, 10660.0, 15950.0, 1083118.0, 6959.0, 20019.0, 35787.0))",0.6017094596028741
0.0,0.0,5983,179490,1660,2280,2550,244294.0,2011.0,7647.0,6364.0,"List(1, 8, List(), List(179490.0, 1660.0, 2280.0, 2550.0, 244294.0, 2011.0, 7647.0, 6364.0))",0.5099302316793666
0.0,0.0,6765,202950,1160,1050,1280,123276.0,1366.0,4671.0,4683.0,"List(1, 8, List(), List(202950.0, 1160.0, 1050.0, 1280.0, 123276.0, 1366.0, 4671.0, 4683.0))",0.4736123710194217
0.0,0.0,7313,263268,19740,14960,28140,2222300.0,13873.0,33345.0,58180.0,"List(1, 8, List(), List(263268.0, 19740.0, 14960.0, 28140.0, 2222300.0, 13873.0, 33345.0, 58180.0))",0.38023552876509303
1.0,211.6,0,0,0,0,0,0.0,0.0,0.0,0.0,"List(0, 8, List(), List())",0.7877943753218855
1.0,875.2,0,0,0,0,0,0.0,0.0,0.0,0.0,"List(0, 8, List(), List())",0.7877943753218855
1.0,2350.4,0,0,0,0,0,0.0,0.0,0.0,0.0,"List(0, 8, List(), List())",0.7877943753218855
1.0,2815.0,2815,50670,8510,7970,15820,916019.0,5718.0,11020.0,22524.0,"List(1, 8, List(), List(50670.0, 8510.0, 7970.0, 15820.0, 916019.0, 5718.0, 11020.0, 22524.0))",0.7093533693696518
1.0,4968.6,0,0,0,0,0,0.0,0.0,0.0,0.0,"List(0, 8, List(), List())",0.7877943753218855


Contrasted with linear regression where the output is assumed to follow a Gaussian distribution, generalized linear models (GLMs) are specifications of linear models where the response variable Yi follows some distribution from the exponential family of distributions. Spark’s GeneralizedLinearRegression interface allows for flexible specification of GLMs which can be used for various types of prediction problems including linear regression, Poisson regression, logistic regression, and others. 
Reference: https://spark.apache.org/docs/2.2.0/ml-classification-regression.html#generalized-linear-regression

In [21]:
glrReg = GeneralizedLinearRegression (family='gaussian', link='identity', maxIter=50, regParam=0.3, labelCol='count')
glrModel = glrReg.fit (train)
glrModelSummary = glrModel.summary
print ("GLR Computed Coefficients = ", glrModel.coefficients)
print ("GLR Computed Intercept = ", glrModel.intercept)
print ("T Values = ", glrModelSummary.tValues)
print ("P Values = ", glrModelSummary.pValues)
print ("GLR Coefficient of Standard Errors = ", glrModelSummary.coefficientStandardErrors)
print ("Dispersion = ", glrModelSummary.dispersion)
print ("Deviance = ", glrModelSummary.deviance)
print ("Null Deviance = ", glrModelSummary.nullDeviance)
print ("Residual Degree of Freedom Null = ", glrModelSummary.residualDegreeOfFreedomNull)
print ("AIC = ", glrModelSummary.aic)

GLR Computed Coefficients = [-2.0972828826491243e-06,3.204769370121298e-07,-2.3976073874504677e-06,-1.4891958386694763e-07,-1.315804703123283e-09,4.4828568980488144e-07,6.8233311937383e-08,2.758819368253426e-08]
GLR Computed Intercept = 0.8333502619029401
T Values = [-33.30236102560992, 0.6395793803062824, -3.1319155935114185, -0.3993899184065202, -0.34147171369080787, 1.7473221143630835, 1.6307442169527953, 0.2557010188413765, 97.40884184361275]
P Values = [0.0, 0.5224643231187867, 0.0017429338103525094, 0.6896165349624312, 0.7327574240270587, 0.08061967584153629, 0.10298349061368928, 0.7981882528495605, 0.0]
GLR Coefficient of Standard Errors = [6.297700277275502e-08, 5.010745294175361e-07, 7.655402311664267e-07, 3.7286765890612536e-07, 3.8533344062422214e-09, 2.5655583828531014e-07, 4.184182364594468e-08, 1.0789238856982625e-07, 0.008555180886360002]
Dispersion = 0.44761706673067975
Deviance = 3597.0507482477424
Null Deviance = 4429.702423865753
Residual Degree of Freedom Null = 8044
AIC = 16375.006797730404

In [22]:
display (glrModelSummary.residuals())

devianceResiduals
-0.7894576495598106
-0.7912674387431808
-0.6848771981461963
-0.45865208550910497
-0.7284227522708872
-0.5075470280829191
-0.3222074258722064
-0.6856052077053332
-0.07171820528913642
-0.354373503194155


In [23]:
glrPredictions = glrModel.transform(test)
glrEvaluator = RegressionEvaluator(labelCol="count", predictionCol="prediction", metricName="rmse")
rmse_glr = glrEvaluator.evaluate(glrPredictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse_glr)

Root Mean Squared Error (RMSE) on test data = 18.1843

In [24]:
display (glrPredictions, 1000)

count,zip,zipcode,sum_zipcode,sum_single_returns,sum_joint_returns,sum_numdep,sum_total_income_amount,sum_taxable_interest_amount,sum_net_capital_gains,sum_biz_net_income,features,prediction
0.0,0.0,305,10980,25040,22560,29630,3711733.0,16160.0,68018.0,128687.0,"List(1, 8, List(), List(10980.0, 25040.0, 22560.0, 29630.0, 3711733.0, 16160.0, 68018.0, 128687.0))",0.7703960447242665
0.0,0.0,2862,120204,9230,10660,15950,1083118.0,6959.0,20019.0,35787.0,"List(1, 8, List(), List(120204.0, 9230.0, 10660.0, 15950.0, 1083118.0, 6959.0, 20019.0, 35787.0))",0.5603204200086129
0.0,0.0,5983,179490,1660,2280,2550,244294.0,2011.0,7647.0,6364.0,"List(1, 8, List(), List(179490.0, 1660.0, 2280.0, 2550.0, 244294.0, 2011.0, 7647.0, 6364.0))",0.45287206995847484
0.0,0.0,6765,202950,1160,1050,1280,123276.0,1366.0,4671.0,4683.0,"List(1, 8, List(), List(202950.0, 1160.0, 1050.0, 1280.0, 123276.0, 1366.0, 4671.0, 4683.0))",0.40626841371482786
0.0,0.0,7313,263268,19740,14960,28140,2222300.0,13873.0,33345.0,58180.0,"List(1, 8, List(), List(263268.0, 19740.0, 14960.0, 28140.0, 2222300.0, 13873.0, 33345.0, 58180.0))",0.2546454785619292
1.0,211.6,0,0,0,0,0,0.0,0.0,0.0,0.0,"List(0, 8, List(), List())",0.8333502619029401
1.0,875.2,0,0,0,0,0,0.0,0.0,0.0,0.0,"List(0, 8, List(), List())",0.8333502619029401
1.0,2350.4,0,0,0,0,0,0.0,0.0,0.0,0.0,"List(0, 8, List(), List())",0.8333502619029401
1.0,2815.0,2815,50670,8510,7970,15820,916019.0,5718.0,11020.0,22524.0,"List(1, 8, List(), List(50670.0, 8510.0, 7970.0, 15820.0, 916019.0, 5718.0, 11020.0, 22524.0))",0.7110746813163363
1.0,4968.6,0,0,0,0,0,0.0,0.0,0.0,0.0,"List(0, 8, List(), List())",0.8333502619029401


We have used the publicly available dataset from the US Government department, and we build our ***Linear Regression (LR)*** and ***Generalized Linear Regression (GLR)*** model on top of it, which we have then used to make predictions on our hold-out or test sample. The RMSE scores were improved from LR to GLR. In the later versions, I plan to include Random Forest and Gradient Boosted Tree on the same dataset for more advanced Data Science use-case. We also realized that we need to evaluate hyper-parameters to achieve the best model. 
The published notebook is available at - *https://databricks-prod-cloudfront.cloud.databricks.com/public/4027ec902e239c93eaaa8714f173bcfc/3173713035751393/175225107118479/2308983777460038/latest.html*